In [1]:
import os
import sys
from dotenv import load_dotenv

from sqlalchemy import create_engine

import pandas as pd


from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

In [2]:
load_dotenv()
database_url = os.getenv('MYSQL_ENGINE_URL')

In [3]:
engine = create_engine(database_url)

In [15]:
query = "SELECT * FROM hotel_booking LIMIT 10000;"

In [16]:
data = pd.read_sql(query, engine)
data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,email,phone-number,credit_card
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,2015-07-01,Ernest Barnes,Ernest.Barnes31@outlook.com,669-792-1661,************4322
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,2015-07-01,Andrea Baker,Andrea_Baker94@aol.com,858-637-6955,************9157
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,Rebecca Parker,Rebecca_Parker@comcast.net,652-885-2745,************3734
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,Laura Murray,Laura_M@gmail.com,364-656-8427,************5677
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,Transient,98.0,0,1,Check-Out,2015-07-03,Linda Hines,LHines@verizon.com,713-226-5883,************5498


In [17]:
# Dropping the features
data = data.drop(['reservation_status',
              'reservation_status_date',
              'assigned_room_type', 
              'arrival_date_year', 
              'country', 
              'agent', 
              'company', 
              'name', 
              'email', 
              'phone-number', 
              'credit_card'], 
              axis=1)

In [18]:
data.shape

(10000, 25)

In [19]:
data.isna().sum()

hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
booking_changes                   0
deposit_type                      0
days_in_waiting_list              0
customer_type                     0
adr                               0
required_car_parking_spaces       0
total_of_special_requests         0
dtype: int64

In [20]:
# Analyze noisy data
noisy_data = {
    'adr':      data[data['adr'] < 0],
    'adults':   data[data['adults'] == 0],
    'children': data[data['children'] == 10],
    'babies':   data[data['babies'] == 10],
}

noisy_data_count = {key: len(value) for key, value in noisy_data.items()}
noisy_data_count

{'adr': 0, 'adults': 7, 'children': 1, 'babies': 0}

In [21]:
data = data[data['children'] != 10]
data = data[data['adults'] != 0]

In [22]:
# Check if the noisy data has been handled
noisy_data_handled = {
    'adr': data[data['adr'] < 0],
    'adults': data[data['adults'] == 0],
    'children': data[data['children'] == 10],
    'babies': data[data['babies'] == 10],
}

noisy_data_handled_count = {key: len(value) for key, value in noisy_data_handled.items()}
noisy_data_handled_count

{'adr': 0, 'adults': 0, 'children': 0, 'babies': 0}

In [23]:
# categorical features
cat_col = [_ for _ in data.columns if data[_].dtype == 'O']
cat_df = data[cat_col]

# print unique values in categorical features
for i in cat_df.columns:
    print('{} :\n{} \n'.format(i,cat_df[i].unique()))

hotel :
['Resort Hotel'] 

arrival_date_month :
['July' 'August' 'September' 'October' 'November' 'December' 'January'
 'February' 'March' 'April' 'May' 'June'] 

meal :
['BB' 'FB' 'HB' 'SC' 'Undefined'] 

market_segment :
['Direct' 'Corporate' 'Online TA' 'Offline TA/TO' 'Complementary' 'Groups'] 

distribution_channel :
['Direct' 'Corporate' 'TA/TO'] 

reserved_room_type :
['C' 'A' 'D' 'E' 'G' 'F' 'H' 'L'] 

deposit_type :
['No Deposit' 'Refundable' 'Non Refund'] 

customer_type :
['Transient' 'Contract' 'Transient-Party' 'Group'] 



In [38]:
train_df, test_df = train_test_split(data, test_size=0.25, random_state=23)
train_df.shape, test_df.shape

((7494, 25), (2498, 25))

In [39]:
train_df.head()

,hotel,is_canceled,lead_time,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
4754,Resort Hotel,0,169,March,14,31,0,3,2,0.0,...,0,0,A,1,Refundable,0,Transient-Party,66.00,0,0
1399,Resort Hotel,0,32,August,35,28,1,2,2,0.0,...,0,0,A,0,No Deposit,0,Transient,122.00,0,1
439,Resort Hotel,0,7,July,29,16,0,1,2,2.0,...,0,0,C,1,No Deposit,0,Transient,169.00,1,1
1163,Resort Hotel,1,143,August,34,16,2,0,2,0.0,...,0,0,A,0,No Deposit,0,Transient,108.54,0,1
3324,Resort Hotel,1,66,December,49,5,1,1,2,0.0,...,0,0,A,0,No Deposit,0,Transient-Party,64.00,0,0


In [40]:
test_df.head()

,hotel,is_canceled,lead_time,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
699,Resort Hotel,1,31,July,30,25,3,6,2,0.0,...,0,0,D,0,No Deposit,0,Transient,154.0,0,1
4618,Resort Hotel,1,106,March,13,24,1,3,2,0.0,...,0,0,A,0,Non Refund,0,Transient,84.0,0,0
5456,Resort Hotel,1,101,April,18,29,2,5,2,0.0,...,0,0,A,0,No Deposit,0,Transient-Party,47.0,0,0
2564,Resort Hotel,0,0,October,43,22,2,5,2,0.0,...,0,0,A,0,No Deposit,0,Transient,42.5,0,1
5237,Resort Hotel,1,83,April,17,22,2,5,2,0.0,...,0,0,A,0,Non Refund,0,Transient,46.0,0,0


In [41]:
# for preprocessing
one_hot_cols = ['hotel', 'meal', 'market_segment', 'distribution_channel', 'reserved_room_type', 'deposit_type', 'customer_type']
label_cols = ['arrival_date_month']
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [43]:
oh_enc = OneHotEncoder(sparse=False, drop='first')

label_enc = LabelEncoder()

In [42]:
def label_encoding_function(data: pd.DataFrame) -> pd.DataFrame:
    month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    label_cols = ['arrival_date_month']
    columns = label_cols if isinstance(label_cols, list) else [label_cols]
    for col in columns:
        data[col] = data[col].apply(lambda x: month_order.index(x) + 1)
    return data

def onehot_encoding_function(data: pd.DataFrame) -> pd.DataFrame:
    one_hot_cols = ['hotel', 'meal', 'market_segment', 'distribution_channel', 'reserved_room_type', 'deposit_type', 'customer_type']
    data = pd.get_dummies(data, columns=one_hot_cols, drop_first=True)
    data = data.astype(int)
    return data

In [43]:
train_df_encoded = label_encoding_function(train_df)
train_df_encoded = onehot_encoding_function(train_df_encoded)

In [44]:
train_df_encoded.head()

,is_canceled,lead_time,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,...,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
4754,0,169,3,14,31,0,3,2,0,0,...,0,0,0,0,0,0,1,0,0,1
1399,0,32,8,35,28,1,2,2,0,0,...,0,0,0,0,0,0,0,0,1,0
439,0,7,7,29,16,0,1,2,2,0,...,0,0,0,0,0,0,0,0,1,0
1163,1,143,8,34,16,2,0,2,0,0,...,0,0,0,0,0,0,0,0,1,0
3324,1,66,12,49,5,1,1,2,0,0,...,0,0,0,0,0,0,0,0,0,1


In [45]:
scaler = MinMaxScaler()

In [ ]:
train_df_encoded

In [ ]:
def get_preprocessing_functions(self) -> Pipeline: #tuple:
    try:
        # Fetch schema config
        onehot_encoding_columns = one_hot_cols
        label_encoding_columns = label_cols
        scaling_columns = self._schema_config.get('scaling_columns', [])

        # Define individual transformer functions
        def label_encoding_function(data: pd.DataFrame) -> pd.DataFrame:
            month_order = months
            columns = label_encoding_columns if isinstance(label_encoding_columns, list) else [label_encoding_columns]
            for col in columns:
                data[col] = data[col].apply(lambda x: month_order.index(x) + 1)
            return data

        def onehot_encoding_function(data: pd.DataFrame) -> pd.DataFrame:
            data = pd.get_dummies(data, columns=onehot_encoding_columns, drop_first=True)
            data = data.astype(int)
            return data
        
        def scaling_function(data: pd.DataFrame) -> pd.DataFrame:
            data[scaling_columns] = scaler.fit_transform(data[scaling_columns])
            return data

        # Initialize transformers
        label_encoder = FunctionTransformer(label_encoding_function)
        onehot_encoder = FunctionTransformer(onehot_encoding_function)
        scaler = MinMaxScaler()

        # Combine transformers in ColumnTransformer
        transformers = []
        if label_encoding_columns:
            transformers.append(('label_encoder', label_encoder, label_encoding_columns))
        if onehot_encoding_columns:
            transformers.append(('onehot_encoder', onehot_encoder, onehot_encoding_columns))
        if scaling_columns:
            transformers.append(('scaler', scaler, scaling_columns))
        
        preprocessor = ColumnTransformer(transformers=transformers, remainder='passthrough')

        # Create pipeline
        data_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])


        # Save the pipeline
        # save_object(self.data_preprocessing_config.preprocessed_object_file_path, data_pipeline)

        return data_pipeline
        # return label_encoding_function, onehot_encoding_function, scaling_function

    except Exception as e:
        raise BaseException(f"Error in get_data_preprocessor_object: {str(e)}", sys) from e
    